In [1]:

from fairseq.tasks.denoising import DenoisingTask
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
import os
import torch

from fairseq.tasks.masked_lm import MaskedLMTask

checkpoint_path = "/data/home/xwhan/fairseq-py/checkpoints/roberta.base"
dictionary = MaskedLMTask.load_dictionary(os.path.join(checkpoint_path, 'dict.txt'))
state = torch.load(os.path.join(checkpoint_path, 'model.pt'), map_location=torch.device('cpu'))
roberta_cfg = convert_namespace_to_omegaconf(state['args'])
task = MaskedLMTask(state['args'], dictionary)
roberta = task.build_model(roberta_cfg.model)
roberta.load_state_dict(state['model'], strict=True, model_cfg=roberta_cfg.model)


bart_path = "/data/home/xwhan/fairseq-py/checkpoints/bart.large"
dictionary = DenoisingTask.load_dictionary(os.path.join(bart_path, 'dict.txt'))
state = torch.load(os.path.join(bart_path, 'model.pt'), map_location=torch.device('cpu'))
bart_cfg = convert_namespace_to_omegaconf(state['args'])
task = DenoisingTask(state['args'], dictionary)
bart = task.build_model(bart_cfg.model)
bart.load_state_dict(state['model'], strict=True, model_cfg=bart_cfg.model)


<All keys matched successfully>

In [2]:
from fairseq.tasks.model_based_denoising import ModelDenoisingTask

model_args = state['args']
model_args.use_xformers = True
model_args.attention_name = 'blocksparse_local'
model_args.xformer_config = '{"block_size": 512, "seq_len": 8192}'
model_args.max_source_positions = 1024 * 8
model_args.max_target_positions = 1024
model_args.mean_noise_span_length = 3
model_args.noise_density = 0.15
model_args.sample_ratio = 0.5
model_args.tokens_per_sample = 1024 * 8

task = ModelDenoisingTask(model_args, dictionary)

long_cfg = convert_namespace_to_omegaconf(model_args)
long_model = task.build_model(long_cfg.model)

2022-02-18 19:06:30 | INFO | fairseq.tasks.model_based_denoising | Adding 334971 new sentinel tokens
2022-02-18 19:06:37 | WARNING | root | blocksparse_local not available among dict_keys(['favor', 'global', 'linformer', 'local', 'nystrom', 'orthoformer', 'random', 'scaled_dot_product', 'block_noglobal', 'fourier_mix', 'compositional', 'block_native', 'lambda', 'block', 'longformer', 'blockwise', 'lsh_global', 'performer', 'nystrom_orig', 'sinkhorn', 'simple_block', 'blockwise_noglobal', 'longshort', 'lsh_reformer'])


KeyError: 'blocksparse_local'